In [4]:
import networkx as nx
import nb_general as nb
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
from math import log2, log

In [40]:
def ApproxRow(v,eps,rho,alpha,n):
    """
    v - node
    eps - additive error parameters 0 < eps < 1
    rho - multiplicative approximation parameter 0 < rho < 1
    alpha - teleportation probability 0 < alpha < 1
    n - number of nodes
    """
    # Initialize dictionary
    NodeCountTree = {key: 0 for key in list(range(n))}
    
    # Set limits
    length = np.int(log(4/eps, 1/(1-alpha)))
    r = np.int(1/(eps*rho**2)*16*log2(n))
    
    # Take r random walks
    for _ in range(r):
        # Initiate random walk
        terminate = False
        cur = v
        i = 0
        
        # Walk
        while terminate is False:
            i += 1
            p = np.random.random()
            
            # With prob 1-alpha take step
            if p <= 1-alpha:
                neighbors = list(G.neighbors(cur))
                cur = np.random.choice(neighbors)
            
            # With prob alpha terminate
            else:
                terminate = True
            
            # Check length
            if i == length:
                terminate = True
        
        # Add one to count if terminated at cur
        NodeCountTree[cur] += 1
    
    # Get average count
    for u in range(n):
        NodeCountTree[u] = NodeCountTree[u]/r
    
    return NodeCountTree

In [53]:
G = nx.gnp_random_graph(100,.7)

In [54]:
ApproxRow(0,2**(-3),1/2,.1,100)

{0: 0.1152602175830638,
 1: 0.009997059688326962,
 2: 0.012349309026756836,
 3: 0.009703028521023229,
 4: 0.007056748015289621,
 5: 0.0111731843575419,
 6: 0.007056748015289621,
 7: 0.009703028521023229,
 8: 0.010879153190238165,
 9: 0.010879153190238165,
 10: 0.007644810349897089,
 11: 0.011761246692149369,
 12: 0.006468685680682152,
 13: 0.0073507791825933545,
 14: 0.010291090855630698,
 15: 0.006762716847985887,
 16: 0.006762716847985887,
 17: 0.00911496618641576,
 18: 0.008526903851808291,
 19: 0.010291090855630698,
 20: 0.009408997353719493,
 21: 0.0073507791825933545,
 22: 0.00911496618641576,
 23: 0.009408997353719493,
 24: 0.006468685680682152,
 25: 0.00911496618641576,
 26: 0.006762716847985887,
 27: 0.009997059688326962,
 28: 0.012643340194060571,
 29: 0.00558659217877095,
 30: 0.008820935019112026,
 31: 0.012055277859453102,
 32: 0.008232872684504558,
 33: 0.008232872684504558,
 34: 0.009703028521023229,
 35: 0.009408997353719493,
 36: 0.006762716847985887,
 37: 0.0108791531

In [ ]:
def ApproximatePageRank(delta, G):
    """
    delta - pagerank threshold
    G - network
    """
    # Initialize variables
    ChunkTree = {}
    n = len(G.nodes())
    
    for t in range(int(log2(n/(4*delta)))):
        # Set additive error
        e_t = 2**(-t)
        
        for _ in range(int(n/delta*e_t*log2(n)**2)):
            # Jump to a random node
            v = np.random.choice(n)
            
            # Get call list
            call_list = ApproxRow(v,e_t/2,1/2)
            
            for node in call_list:
                if (node,e_t) in list(ChunkTree.keys()):
                    ChunkTree